In [ ]:
# !pip install xlrd

In [ ]:
import pandas as pd
import numpy as np

Here we're going to use the read excel functionality of Pandas. The `sheet_name=None` means that we want to include all the sheets in the document.

In [ ]:
tkm_data = pd.read_excel('./KA_TKM_2019-08-10.xlsx',sheet_name=None)

If you take the type of the variable, you find something different than usual, but it's not completely foreign to us. Look at the last part:

In [ ]:
type(tkm_data)

read_excel returned a type of a dictionary. We know what dictionaries have: keys and values. Let's look at the keys:

In [ ]:
tkm_data.keys()

Since these keys are attached to values, I wonder what `type()` we see if we use the `.get(key)` method:

In [ ]:
type(tkm_data.get('KA-Tags'))

It appears that this new object is a _dictionary_ of _dataframes_ ! Since all of the values are dataframes, we can do dataframe stuff with them:

In [ ]:
tkm_data.get('KA-Tags').info()

For ease of typing, let's just create a new variable and point it at the dataframe in the dictionary:

In [ ]:
ka_tags = tkm_data.get('KA-Tags')

Let's start looking at what the database contains:

In [ ]:
ka_tags.head()

The 'Key Activity' field has some codes paired with descriptions. I wonder if those pairs are always consistent. First, let's look at how many unique values are in that field. We're going to use the `.unique()` to return a numpy array of values and then look at the size.

In [ ]:
ka_tags['Key Activity'].unique().size

Just to introduce some other ways to think about numpy arrays, they can themselves be passed as arguments to a function. This is a numpy function that counts the non-zero values:

In [ ]:
np.count_nonzero(ka_tags['Key Activity'].unique())

Now, to investigate the codes and the descriptions separate from each other. Let's see if we can see what an example might be: 

In [ ]:
ka_tags.loc[225,'Key Activity']

Looking at this, it seems that the code at the front is separated from the description at the end with a space-hyphen-space composite delimiter. I wonder if we could _split_ using that:

In [ ]:
ka_tags['Key Activity'].str.split(' - ').head(10)

Split on multiple consecutive characters seems to work without an error. We've now created a bunch of lists. We can use the apply method to convert these lists to series in place:

In [ ]:
ka_tags['Key Activity'].str.split(' - ').apply(pd.Series).head()

That third column is a little weird. It probably means that there is an additional space-hyphen-space in the text on a few of these. No problem! Let's just split on the first occurrence, by passing an additional argument to the `.split()` method:

In [ ]:
ka_tags['Key Activity'].str.split(' - ',1).apply(pd.Series).head()

This looks better. Codes in column 0, descriptions in column 1 (hopefully). But is this the best way to do this? There are a couple of ways to convert lists into when splitting on a character. How do we determine the best one? Thankfully, Python has the 'timeit' command. Using it, we can run a segment of code numerous times, and see how time efficent it is. We can do the same thing with another code segment and compare the two. Check this out: 

In [ ]:
%timeit ka_tags['Key Activity'].str.split(' - ',1).apply(pd.Series)

The split method in Pandas can take an additional argument: `expand`. This will take the results of the split operation (one or more lists) and convert them to series in place. If we use `%timeit` again, we can see if it is better:

In [ ]:
%timeit ka_tags['Key Activity'].str.split(' - ',1,expand=True)

wow! That is a reduction of upwards of 90%! In this particular case, we don't see much of a difference, as the dataset is relatively small. However, if you're trying to do transformations on significantly larger datasets, a reduction of this size would be tremendous.

let's make sure that the operations are equivalent, just to be sure:

In [ ]:
np.all((ka_tags['Key Activity'].str.split(' - ',1).apply(pd.Series)) == (ka_tags['Key Activity'].str.split(' - ',1,expand=True)))

Moving forward, let's use the more efficient method and check on our results. To start, we'll see how many unique codes there are. Since we're creating a DataFrame, we can use our indexers to separate the columns. Afterward, we'll use `.unique().size` to count the number of unique values in this column.

In [ ]:
ka_tags['Key Activity'].str.split(' - ',1,expand=True).iloc[:,0].unique().size

That's a different number from above. I wonder how many unique descriptions there are:

In [ ]:
ka_tags['Key Activity'].str.split(' - ',1,expand=True).iloc[:,1].unique().size

Maybe there are supposed to be codes that are different from the associated descriptions. In case there aren't, though, let's `.groupby()` the two columns and count them, ordered by code:

In [ ]:
ka_tags['Key Activity'].str.split(' - ',1,expand=True).groupby([0,1]).size().reset_index(name='Count')

We can already see pairs that don't go with their codes. What if we reverse the order, and sort by description?

In [ ]:
ka_tags['Key Activity'].str.split(' - ',1,expand=True).groupby([1,0]).size().reset_index(name='Count')

Ordering the data in this way shows a couple of situations where missing words or descriptions might be read by a human as similar, but would be considered very different by a computer:

In [ ]:
ka_tags['Key Activity'].str.split(' - ',1,expand=True).groupby([1,0]).size().reset_index(name='Count').iloc[222,0]

In [ ]:
ka_tags['Key Activity'].str.split(' - ',1,expand=True).groupby([1,0]).size().reset_index(name='Count').iloc[223,0]

In [ ]:
ka_tags['Key Activity'].str.split(' - ',1,expand=True).groupby([1,0]).size().reset_index(name='Count').iloc[224,0]

In [ ]:
ka_tags['Key Activity'].str.split(' - ',1,expand=True).groupby([1,0]).size().reset_index(name='Count').iloc[225,0]

In [ ]:
ka_tags['Key Activity'].str.split(' - ',1,expand=True).groupby([1,0]).size().reset_index(name='Count').iloc[18,0]

In [ ]:
ka_tags['Key Activity'].str.split(' - ',1,expand=True).groupby([1,0]).size().reset_index(name='Count').iloc[19,0]

Now, let's create a new DataFrame with our split text, and concatenate it on the existing DataFrame.

In [ ]:
ka_tags = pd.concat([ka_tags,ka_tags['Key Activity'].str.split(' - ',1,expand=True)],axis=1).rename(columns={0:'code',1:'name'})

Now that we're aware that there are some inconsistencies in these fields, let's work on cleaning them up. We'll start by eliminating one of the biggest traps in messy data - the repeated whitespace.

In [ ]:
ka_tags['name'].unique().size

In [ ]:
ka_tags['name'].str.replace('\s{2,}',' ').unique().size

Ok. So that isn't really that impressive of change. But, remember, to a computer, `tuna fish` and `tuna  fish` are different strings. Hey, it's a start.

In [ ]:
ka_tags['name'] = ka_tags['name'].str.replace('\s+',' ')

In [ ]:
from fuzzywuzzy import fuzz, process

In [ ]:
ka_tags[ka_tags['name'].str.contains('high-quality')]['name']

In [ ]:
ka_tags.loc[931,'name']

In [ ]:
process.extract(ka_tags.loc[931,'name'],ka_tags['name'].unique())